## 用训练好的caffemodel来进行分类

caffe程序自带有一张小猫图片，存放路径为caffe根目录下的 examples/images/cat.jpg, 如果我们想用一个训练好的caffemodel来对这张图片进行分类，那该怎么办呢？ 如果不用这张小猫图片，换一张别的图片，又该怎么办呢？如果学会了小猫图片的分类，那么换成其它图片，程序实际上是一样的。

开发caffe的贾大牛团队，利用imagenet图片和caffenet模型训练好了一个caffemodel,  供大家下载。要进行图片的分类，这个caffemodel是最好不过的了。所以，不管是用c++来进行分类，还是用python接口来分类，我们都应该准备这样三个文件：

1、caffemodel文件。 

  可以直接在浏览器里输入地址下载，也可以运行脚本文件下载。下载地址为：http://dl.caffe.berkeleyvision.org/bvlc_reference_caffenet.caffemodel

文件名称为：bvlc_reference_caffenet.caffemodel，文件大小为230M左右，为了代码的统一，将这个caffemodel文件下载到caffe根目录下的 models/bvlc_reference_caffenet/ 文件夹下面。也可以运行脚本文件进行下载：


In [11]:
! pwd

/home/yeler082/caffe


In [12]:
import sys,os,caffe
#设置当前目录
caffe_root = '/home/yeler082/caffe/' 
# sys.path.insert(0, caffe_root + 'python')
os.chdir(caffe_root)

In [13]:
! pwd

/home/yeler082/caffe


In [14]:
! ./scripts/download_model_binary.py models/bvlc_reference_caffenet

Model already exists.


2、均值文件。

有了caffemodel文件，就需要对应的均值文件，在测试阶段，需要把测试数据减去均值。这个文件我们用脚本来下载，在caffe根目录下执行：

In [15]:
! sh ./data/ilsvrc12/get_ilsvrc_aux.sh

Downloading...
--2018-10-20 09:33:13--  http://dl.caffe.berkeleyvision.org/caffe_ilsvrc12.tar.gz
Resolving dl.caffe.berkeleyvision.org (dl.caffe.berkeleyvision.org)... 169.229.222.251
Connecting to dl.caffe.berkeleyvision.org (dl.caffe.berkeleyvision.org)|169.229.222.251|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://202.117.4.101/cache/6/02/dl.caffe.berkeleyvision.org/7dc6c6cb703abfdad189a236c0d3d32e/caffe_ilsvrc12.tar.gz [following]
--2018-10-20 09:33:13--  http://202.117.4.101/cache/6/02/dl.caffe.berkeleyvision.org/7dc6c6cb703abfdad189a236c0d3d32e/caffe_ilsvrc12.tar.gz
Connecting to 202.117.4.101:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17858008 (17M) [application/octet-stream]
Saving to: ‘caffe_ilsvrc12.tar.gz’

caffe_ilsvrc12.tar. 100%[===================>]  17.03M  88.7MB/s    in 0.2s    

2018-10-20 09:33:13 (88.7 MB/s) - ‘caffe_ilsvrc12.tar.gz’ saved [17858008/17858008]

Unzipping...
Done.


执行并下载后，均值文件放在 data/ilsvrc12/ 文件夹里。

3、synset_words.txt文件

在调用脚本文件下载均值的时候，这个文件也一并下载好了。里面放的是1000个类的名称。

数据准备好了，我们就可以开始分类了，我们给大家提供两个版本的分类方法：

### 一、c++方法

在caffe根目录下的 examples/cpp-classification/ 文件夹下面，有个classification.cpp文件，就是用来分类的。当然编译后，放在/build/examples/cpp_classification/ 下面

我们就直接运行命令：

In [16]:
! ./build/examples/cpp_classification/classification.bin \
  models/bvlc_reference_caffenet/deploy.prototxt \
  models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel \
  data/ilsvrc12/imagenet_mean.binaryproto \
  data/ilsvrc12/synset_words.txt \
  examples/images/cat.jpg

---------- Prediction for examples/images/cat.jpg ----------
0.3134 - "n02123045 tabby, tabby cat"
0.2380 - "n02123159 tiger cat"
0.1235 - "n02124075 Egyptian cat"
0.1003 - "n02119022 red fox, Vulpes vulpes"
0.0715 - "n02127052 lynx, catamount"


命令很长，用了很多的\符号来换行。可以看出，从第二行开始就是参数，每行一个，共需要4个参数

运行成功后，输出top-5结果：
即有0.3134的概率为tabby cat, 有0.2380的概率为tiger cat ......

### 二、python方法

python接口可以使用jupyter notebook来进行可视化操作，因此推荐使用这种方法。

在这里我就不用可视化了，编写一个py文件，命名为py_classify.py

In [17]:
#coding=utf-8
#加载必要的库
import numpy as np
import sys,os

#设置当前目录
caffe_root = '/home/yeler082/caffe/' 
sys.path.insert(0, caffe_root + 'python')
import caffe
os.chdir(caffe_root)

net_file=caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
caffe_model=caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
mean_file=caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy'

net = caffe.Net(net_file,caffe_model,caffe.TEST)
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.load(mean_file).mean(1).mean(1))
transformer.set_raw_scale('data', 255) 
transformer.set_channel_swap('data', (2,1,0))

im=caffe.io.load_image(caffe_root+'examples/images/cat.jpg')
net.blobs['data'].data[...] = transformer.preprocess('data',im)
out = net.forward()


imagenet_labels_filename = caffe_root + 'data/ilsvrc12/synset_words.txt'
labels = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')

top_k = net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
for i in np.arange(top_k.size):
    print top_k[i], labels[top_k[i]]

281 n02123045 tabby, tabby cat
282 n02123159 tiger cat
285 n02124075 Egyptian cat
277 n02119022 red fox, Vulpes vulpes
287 n02127052 lynx, catamount


caffe开发团队实际上也编写了一个python版本的分类文件，路径为 python/classify.py

运行这个文件必需两个参数，一个输入图片文件，一个输出结果文件。而且运行必须在python目录下。假设当前目录是caffe根目录，则运行：

In [20]:
os.chdir(caffe_root+'/python')
! python2 classify.py ../examples/images/cat.jpg result.npy

CPU mode
I1020 09:40:32.330868  6533 net.cpp:51] Initializing net from parameters: 
name: "CaffeNet"
state {
  phase: TEST
  level: 0
}
layer {
  name: "data"
  type: "Input"
  top: "data"
  input_param {
    shape {
      dim: 10
      dim: 3
      dim: 227
      dim: 227
    }
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 96
    kernel_size: 11
    stride: 4
  }
}
layer {
  name: "relu1"
  type: "ReLU"
  bottom: "conv1"
  top: "conv1"
}
layer {
  name: "pool1"
  type: "Pooling"
  bottom: "conv1"
  top: "pool1"
  pooling_param {
    pool: MAX
    kernel_size: 3
    stride: 2
  }
}
layer {
  name: "norm1"
  type: "LRN"
  bottom: "pool1"
  top: "norm1"
  lrn_param {
    local_size: 5
    alpha: 0.0001
    beta: 0.75
  }
}
layer {
  name: "conv2"
  type: "Convolution"
  bottom: "norm1"
  top: "conv2"
  convolution_param {
    num_output: 256
    pad: 2
    kernel_size: 5
    group: 2
  }
}
layer {
  name: "relu2"

I1020 09:40:32.952467  6533 net.cpp:122] Setting up conv4
I1020 09:40:32.952482  6533 net.cpp:129] Top shape: 10 384 13 13 (648960)
I1020 09:40:32.952487  6533 net.cpp:137] Memory required for data: 61192760
I1020 09:40:32.952494  6533 layer_factory.hpp:77] Creating layer relu4
I1020 09:40:32.952502  6533 net.cpp:84] Creating Layer relu4
I1020 09:40:32.952505  6533 net.cpp:406] relu4 <- conv4
I1020 09:40:32.952512  6533 net.cpp:367] relu4 -> conv4 (in-place)
I1020 09:40:32.953363  6533 net.cpp:122] Setting up relu4
I1020 09:40:32.953377  6533 net.cpp:129] Top shape: 10 384 13 13 (648960)
I1020 09:40:32.953380  6533 net.cpp:137] Memory required for data: 63788600
I1020 09:40:32.953400  6533 layer_factory.hpp:77] Creating layer conv5
I1020 09:40:32.953410  6533 net.cpp:84] Creating Layer conv5
I1020 09:40:32.953415  6533 net.cpp:406] conv5 <- conv4
I1020 09:40:32.953421  6533 net.cpp:380] conv5 -> conv5
I1020 09:40:32.957916  6533 net.cpp:122] Setting up conv5
I1020 09:40:32.957929  6533

I1020 09:40:33.588443  6533 upgrade_proto.cpp:63] Successfully upgraded file specified using deprecated V1LayerParameter
I1020 09:40:33.637914  6533 net.cpp:744] Ignoring source layer loss
Loading file: ../examples/images/cat.jpg
Classifying 1 inputs.
Done in 1.17 s.
Saving results into result.npy
